In [47]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

### Importing the review files 
There are two different files cointaining reviews. One file have pos reviews and the other have neg reviews

In [48]:
pos_rev = pd.read_csv('netflix/pos.txt' , sep = '\n' , header= None , encoding='latin-1')
pos_rev['mood'] = 1
pos_rev.rename(columns = {0: 'review'}, inplace = True)
pos_rev.head()

,review,mood
0,the rock is destined to be the 21st century's ...,1
1,"the gorgeously elaborate continuation of "" the...",1
2,effective but too-tepid biopic,1
3,if you sometimes like to go to the movies to h...,1
4,"emerges as something rare , an issue movie tha...",1


In [49]:
neg_rev = pd.read_csv('netflix/negative.txt' , sep = '\n' , header= None , encoding='latin-1')
neg_rev['mood'] = 0
neg_rev.rename(columns = {0: 'review'}, inplace = True)
neg_rev.head()

,review,mood
0,"simplistic , silly and tedious.",0
1,"it's so laddish and juvenile , only teenage bo...",0
2,exploitative and largely devoid of the depth o...,0
3,[garbus] discards the potential for pathologic...,0
4,a visually flashy but narratively opaque and e...,0


In [50]:
df = pd.concat([pos_rev, neg_rev]).reset_index(drop = True)
df.head()

,review,mood
0,the rock is destined to be the 21st century's ...,1
1,"the gorgeously elaborate continuation of "" the...",1
2,effective but too-tepid biopic,1
3,if you sometimes like to go to the movies to h...,1
4,"emerges as something rare , an issue movie tha...",1


In [51]:
def clean(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [52]:
df['review'] = df['review'].apply(lambda x: clean(x))

In [53]:
df.head()

,review,mood
0,the rock is destined to be the centurys new ...,1
1,the gorgeously elaborate continuation of the ...,1
2,effective but tootepid biopic,1
3,if you sometimes like to go to the movies to h...,1
4,emerges as something rare an issue movie that...,1


In [54]:
xtrain,xtest,ytrain,ytest = train_test_split(df['review'], df['mood'], random_state = 44, test_size = 0.2)

In [55]:
vectorizer = TfidfVectorizer()
train_vec = vectorizer.fit_transform(xtrain)
test_vec  = vectorizer.transform(xtest)

In [56]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report
classifier  = SVC(kernel = 'linear')
classifier.fit(train_vec, ytrain)
y_pred = classifier.predict(test_vec)
print(classification_report(ytest, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.76      0.75      1066
           1       0.75      0.74      0.75      1067

    accuracy                           0.75      2133
   macro avg       0.75      0.75      0.75      2133
weighted avg       0.75      0.75      0.75      2133



In [57]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(train_vec,ytrain)
y_pred_clf = clf.predict(test_vec)
print(classification_report(ytest, y_pred_clf))

              precision    recall  f1-score   support

           0       0.76      0.77      0.77      1066
           1       0.77      0.75      0.76      1067

    accuracy                           0.76      2133
   macro avg       0.76      0.76      0.76      2133
weighted avg       0.76      0.76      0.76      2133



In [58]:
import joblib
joblib.dump(clf,'naive_clf.pkl')
joblib.dump(vectorizer, 'transform.pkl')

['transform.pkl']